## 基础对话模型搭建

本节基于上一章的 Transformer 基础，系统讲解如何把通用的 `Transformer` 模型改进为适应本项目的共情对话模型：在不破坏原有 Encoder/Decoder 主干的前提下，加入情感分类头与相应的训练目标，并对接情感词典，形成“情感引导”的对话生成。

注：以下所有内容，仅包含核心代码供讲解所需。完整代码内容较多，请阅读源码查看。

代码包括：
- ./Model/common_layer.py
- ./Model/EmoPrepend.py

### 1. 通用模块封装（common_layer.py）

我们在 `common_layer.py` 中实现了 Transformer 及相关通用模块的核心文件，主要包括以下几大部分：

1. 全局初始化与依赖  
2. EncoderLayer / GraphLayer / DecoderLayer  
3. MultiHeadAttention (多头注意力)  
4. Conv 与 PositionwiseFeedForward
5. LayerNorm  
6. 掩码与位置编码工具函数  
7. Embedding 相关  
8. LabelSmoothing  
9. NoamOpt（学习率调度器）

以下文档中仅展示最核心的代码模块。

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.nn.init as I
import numpy as np
import math
import os
import pdb
import numpy as np

#### 1.1 多头注意力机制
MultiHeadAttention (多头注意力) 包含：
   - 线性映射：query/key/value → 划分多头 → 缩放点积 → Mask（可选）  
   - Softmax 归一化 → Dropout → 拼回各头输出 → 线性投影  
   - 返回 `(outputs, attention_weights)`，其中 `attention_weights` 是跨头平均后的注意力权重  

In [2]:
class MultiHeadAttention(nn.Module):
    def __init__(self, input_depth, total_key_depth, total_value_depth, output_depth,
                 num_heads, bias_mask=None, dropout=0.0):
        """
        参数:
            input_depth (int): 输入张量最后一维维度
            total_key_depth (int): 键张量最后一维维度，需被 num_heads 整除
            total_value_depth (int): 值张量最后一维维度，需被 num_heads 整除
            output_depth (int): 输出张量最后一维维度
            num_heads (int): 注意力头数量
            bias_mask (Tensor or None): 用于屏蔽未来位置的掩码
            dropout (float): 注意力权重上的 Dropout 概率（训练时非零）
        """
        super(MultiHeadAttention, self).__init__()

        if total_key_depth % num_heads != 0:
            print("Key depth (%d) must be divisible by the number of "
                  "attention heads (%d)." % (total_key_depth, num_heads))
            total_key_depth = total_key_depth - (total_key_depth % num_heads)
        if total_value_depth % num_heads != 0:
            print("Value depth (%d) must be divisible by the number of "
                  "attention heads (%d)." % (total_value_depth, num_heads))
            total_value_depth = total_value_depth - (total_value_depth % num_heads)
        self.num_heads = num_heads
        self.query_scale = (total_key_depth // num_heads) ** -0.5
        self.bias_mask = bias_mask
        self.query_linear = nn.Linear(input_depth, total_key_depth, bias=False)
        self.key_linear = nn.Linear(input_depth, total_key_depth, bias=False)
        self.value_linear = nn.Linear(input_depth, total_value_depth, bias=False)
        self.output_linear = nn.Linear(total_value_depth, output_depth, bias=False)
        self.emotion_output_linear = nn.Linear(2 * output_depth, output_depth, bias=False)
        self.W_vad = nn.Parameter(torch.FloatTensor(1))
        self.dropout = nn.Dropout(dropout)

    def _split_heads(self, x):
        """
        拆分多头：在特征维度上增加 num_heads 维度
        输入:
            x: 形状为 [batch_size, seq_length, depth] 的张量
        返回:
            形状为 [batch_size, num_heads, seq_length, depth/num_heads] 的张量
        """
        if len(x.shape) != 3:
            raise ValueError("x must have rank 3")
        shape = x.shape
        return x.view(shape[0], shape[1], self.num_heads, shape[2] // self.num_heads).permute(0, 2, 1, 3)

    def _merge_heads(self, x):
        """
        合并多头：将 num_heads 维度合并回最后一维
        输入:
            x: 形状为 [batch_size, num_heads, seq_length, depth/num_heads] 的张量
        返回:
            形状为 [batch_size, seq_length, depth] 的张量
        """
        if len(x.shape) != 4:
            raise ValueError("x must have rank 4")
        shape = x.shape
        return x.permute(0, 2, 1, 3).contiguous().view(shape[0], shape[2], shape[3] * self.num_heads)

    def forward(self, queries, keys, values, mask):
        # 对查询、键、值分别进行线性映射
        queries = self.query_linear(queries)
        keys = self.key_linear(keys)
        values = self.value_linear(values)

        # 拆分多头
        queries = self._split_heads(queries)  # (bsz, heads, len, key_depth-20)
        keys = self._split_heads(keys)
        values = self._split_heads(values)

        # 缩放查询向量
        queries *= self.query_scale
        # 计算注意力得分
        logits = torch.matmul(queries, keys.permute(0, 1, 3, 2))  # (bsz, head, tgt_len, src_len)

        if mask is not None:
            mask = mask.unsqueeze(1)  # 扩展掩码维度至 [B, 1, 1, T_values]
            logits = logits.masked_fill(mask, -1e18)

        # 计算整体注意力权重 (平均各头)
        attetion_weights = logits.sum(dim=1) / self.num_heads  # (bsz, tgt_len, src_len)
        # 将得分转换为概率分布
        weights = nn.functional.softmax(logits, dim=-1)  # (bsz, 2, tgt_len, src_len)
        # 对注意力概率进行 Dropout
        weights = self.dropout(weights)
        # 与值向量相乘，获取上下文表示
        contexts = torch.matmul(weights, values)
        # 合并多头，恢复上下文向量形状
        contexts = self._merge_heads(contexts)
        # contexts = torch.tanh(contexts)
        # 线性变换输出
        outputs = self.output_linear(contexts)  # 50 -> 300
        # 返回输出及注意力权重（已归一化）
        return outputs, torch.softmax(attetion_weights, dim=-1)

#### 1.2 编码器层

EncoderLayer 包含：
- 自注意力（MultiHeadAttention）+ 前馈网络（PositionwiseFeedForward）  
- 每个子层前都做 LayerNorm，子层输出后做残差连接和 Dropout  

In [4]:
from Model.common_layer import Conv, PositionwiseFeedForward, LayerNorm, MultiHeadAttention


class EncoderLayer(nn.Module):
    """
    Transformer 编码器（Encoder）中的单层结构。
    参考论文：https://arxiv.org/pdf/1706.03762.pdf
    """
    def __init__(self, hidden_size, total_key_depth, total_value_depth, filter_size, num_heads,
                 bias_mask=None, layer_dropout=0.0, attention_dropout=0.0, relu_dropout=0.0):
        """
        参数:
            hidden_size (int): 隐藏层维度大小
            total_key_depth (int): 键（Key）张量最后一维的大小，需能被 num_heads 整除
            total_value_depth (int): 值（Value）张量最后一维的大小，需能被 num_heads 整除
            output_depth (int): 输出张量最后一维的大小
            filter_size (int): 前馈网络（FFN）中间隐藏层的大小
            num_heads (int): 多头注意力头数
            bias_mask (Tensor or None): 用于屏蔽未来位置的掩码，防止信息泄露
            layer_dropout (float): 本层残差连接后的 Dropout 概率
            attention_dropout (float): 注意力权重上的 Dropout 概率（训练时非零）
            relu_dropout (float): 前馈网络中 ReLU 后的 Dropout 概率（训练时非零）
        """
        super(EncoderLayer, self).__init__()
        self.multi_head_attention = MultiHeadAttention(hidden_size, total_key_depth, total_value_depth,
                                                       hidden_size, num_heads, bias_mask, attention_dropout)
        self.positionwise_feed_forward = PositionwiseFeedForward(hidden_size, filter_size, hidden_size,
                                                                 layer_config='cc', padding='both',
                                                                 dropout=relu_dropout)
        self.dropout = nn.Dropout(layer_dropout)
        self.layer_norm_mha = LayerNorm(hidden_size)
        self.layer_norm_ffn = LayerNorm(hidden_size)

    def forward(self, inputs, mask=None):
        x = inputs
        # 层归一化：对输入进行标准化处理，稳定模型训练
        x_norm = self.layer_norm_mha(x)
        # 多头注意力：执行自注意力计算，融合全局信息
        y, _ = self.multi_head_attention(x_norm, x_norm, x_norm, mask)
        # 残差连接并进行 Dropout
        x = self.dropout(x + y)
        # 层归一化：对残差输出进行标准化
        x_norm = self.layer_norm_ffn(x)
        # 前馈网络：逐位置进行两层线性变换及激活
        y = self.positionwise_feed_forward(x_norm)
        # 残差连接并进行 Dropout
        y = self.dropout(x + y)
        return y

                                      Opts                                      
--------------------------------------------------------------------------------
                                dataset: empathetic                             
                             hidden_dim: 300                                    
                                emb_dim: 300                                    
                             batch_size: 16                                     
                                 epochs: 100                                    
                                     lr: 0.0001                                 
                          max_grad_norm: 2.0                                    
                              beam_size: 5                                      
                              save_path: results/tb_results/test/               
                      save_path_dataset: results/tb_results/                    
                            

#### 1.3 图网络层

GraphLayer中
- 只包含“编码—解码”注意力，用于将编码器输出与图结构信息融合  
- 同样使用 LayerNorm → 注意力 → 残差 → LayerNorm → 前馈 → 残差

encoder_outputs 中通常已经包含了对话上下文及情感图谱的特征，GraphLayer 通过跨注意力将这些图结构／情感信息融合到解码器每一步的表示里，起到图神经网络层（Graph Neural Network Layer）的作用。

In [55]:
class GraphLayer(nn.Module):
    """
    Transformer 解码器（Decoder）中的图网络层（GraphLayer）。
    参考论文：https://arxiv.org/pdf/1706.03762.pdf
    """

    def __init__(self, hidden_size, total_key_depth, total_value_depth, filter_size, num_heads,
                 bias_mask, layer_dropout=0.0, attention_dropout=0.0, relu_dropout=0.0):
        """
        参数:
            hidden_size (int): 隐藏层维度
            total_key_depth (int): Key 张量最后一维维度，需被 num_heads 整除
            total_value_depth (int): Value 张量最后一维维度，需被 num_heads 整除
            filter_size (int): 前馈网络中间层维度
            num_heads (int): 注意力头数量
            bias_mask (Tensor): 屏蔽未来位置的掩码
            layer_dropout (float): 本层残差后的 Dropout 概率
            attention_dropout (float): 注意力权重上的 Dropout 概率
            relu_dropout (float): 前馈网络中 ReLU 后的 Dropout 概率
        """

        super(GraphLayer, self).__init__()
        self.multi_head_attention_enc_dec = MultiHeadAttention(hidden_size, total_key_depth, total_value_depth,
                                                               hidden_size, num_heads, None, attention_dropout)
        self.positionwise_feed_forward = PositionwiseFeedForward(hidden_size, filter_size, hidden_size,
                                                                 layer_config='cc', padding='left',
                                                                 dropout=relu_dropout)
        self.dropout = nn.Dropout(layer_dropout)
        self.layer_norm_mha_dec = LayerNorm(hidden_size)
        self.layer_norm_mha_enc = LayerNorm(hidden_size)
        self.layer_norm_ffn = LayerNorm(hidden_size)

    def forward(self, inputs):
        """
        注：inputs 为元组，包含解码器输入、编码器输出、注意力权重、源掩码
        """
        x, encoder_outputs, attention_weight, mask_src = inputs

        # 在进行编码器-解码器注意力前先做层归一化，稳定训练
        x_norm = self.layer_norm_mha_enc(x)
        # 多头编码器-解码器注意力，融合编码器信息
        y, attention_weight = self.multi_head_attention_enc_dec(x_norm, encoder_outputs, encoder_outputs, mask_src)
        # 编码器-解码器注意力后的残差连接并 Dropout
        x = self.dropout(x + y)
        # 层归一化
        x_norm = self.layer_norm_ffn(x)
        # 前馈网络：逐位置线性 + 激活 + 线性
        y = self.positionwise_feed_forward(x_norm)
        # 前馈网络后的残差连接并 Dropout
        y = self.dropout(x + y)
        return y, encoder_outputs, attention_weight

#### 1.4 解码器层

DecoderLayer中包含
- 标准 Transformer 解码器层：先做掩码自注意力，再做编码—解码注意力，最后做前馈网络
- 每步均包含 LayerNorm、残差连接和 Dropout

In [56]:
class PositionwiseFeedForward(nn.Module):
    """
    对序列中的每个位置先后执行线性变换 + ReLU + 线性变换
    """
    def __init__(self, input_depth, filter_size, output_depth, layer_config='ll', padding='left', dropout=0.0):
        """
        参数:
            input_depth (int): 输入张量最后一维维度
            filter_size (int): 前馈网络中间层维度
            output_depth (int): 输出张量最后一维维度
            layer_config (str): 'll' -> 全线性结构；'cc' -> 卷积结构
            padding (str): 'left' -> 在左侧填充；'both' -> 在两侧填充
            dropout (float): 前馈网络中 Dropout 概率（训练时非零）
        """
        super(PositionwiseFeedForward, self).__init__()
        layers = []
        sizes = ([(input_depth, filter_size)] +
                 [(filter_size, filter_size)] * (len(layer_config) - 2) +
                 [(filter_size, output_depth)])
        for lc, s in zip(list(layer_config), sizes):
            if lc == 'l':
                layers.append(nn.Linear(*s))
            elif lc == 'c':
                layers.append(Conv(*s, kernel_size=3, pad_type=padding))
            else:
                raise ValueError("Unknown layer type {}".format(lc))

        self.layers = nn.ModuleList(layers)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

    def forward(self, inputs):
        x = inputs
        for i, layer in enumerate(self.layers):
            x = layer(x)
            if i < len(self.layers):
                x = self.relu(x)
                x = self.dropout(x)
        return x


class DecoderLayer(nn.Module):
    """
    Transformer 解码器（Decoder）中的单层结构。
    参考论文：https://arxiv.org/pdf/1706.03762.pdf
    """
    def __init__(self, hidden_size, total_key_depth, total_value_depth, filter_size, num_heads,
                 bias_mask, layer_dropout=0.0, attention_dropout=0.0, relu_dropout=0.0):
        """
        参数:
            hidden_size (int): 隐藏层维度
            total_key_depth (int): Key 张量最后一维维度，需被 num_heads 整除
            total_value_depth (int): Value 张量最后一维维度，需被 num_heads 整除
            filter_size (int): 前馈网络中间层维度
            num_heads (int): 注意力头数量
            bias_mask (Tensor): 屏蔽未来位置的掩码
            layer_dropout (float): 本层残差后的 Dropout 概率
            attention_dropout (float): 注意力权重上的 Dropout 概率
            relu_dropout (float): 前馈网络中 ReLU 后的 Dropout 概率
        """
        super(DecoderLayer, self).__init__()
        self.multi_head_attention_dec = MultiHeadAttention(hidden_size, total_key_depth, total_value_depth,
                                                           hidden_size, num_heads, bias_mask, attention_dropout)
        self.multi_head_attention_enc_dec = MultiHeadAttention(hidden_size, total_key_depth, total_value_depth,
                                                               hidden_size, num_heads, None, attention_dropout)
        self.positionwise_feed_forward = PositionwiseFeedForward(hidden_size, filter_size, hidden_size,
                                                                 layer_config='cc', padding='left',
                                                                 dropout=relu_dropout)
        self.dropout = nn.Dropout(layer_dropout)
        self.layer_norm_mha_dec = LayerNorm(hidden_size)
        self.layer_norm_mha_enc = LayerNorm(hidden_size)
        self.layer_norm_ffn = LayerNorm(hidden_size)

    def forward(self, inputs):
        """
        注：inputs 为元组，包含解码器输入、编码器输出、注意力权重、源掩码
        """
        x, encoder_outputs, attention_weight, mask = inputs
        mask_src, dec_mask = mask
        # 在进行解码器自注意力前先做层归一化，稳定训练
        x_norm = self.layer_norm_mha_dec(x)
        # 掩码多头自注意力：对自身序列进行注意力计算，防止未来信息泄露
        y, _ = self.multi_head_attention_dec(x_norm, x_norm, x_norm, dec_mask)
        # 自注意力后的残差连接并 Dropout
        x = self.dropout(x + y)
        # 在进行编码器-解码器注意力前先做层归一化，稳定训练
        x_norm = self.layer_norm_mha_enc(x)
        # 多头编码器-解码器注意力，融合编码器输出
        y, attention_weight = self.multi_head_attention_enc_dec(x_norm, encoder_outputs, encoder_outputs, mask_src)
        # 编码器-解码器注意力后的残差连接并 Dropout
        x = self.dropout(x + y)
        # 层归一化
        x_norm = self.layer_norm_ffn(x)
        # 前馈网络：逐位置两个线性变换及激活
        y = self.positionwise_feed_forward(x_norm)
        # 前馈网络后的残差连接并 Dropout
        y = self.dropout(x + y)
        return y, encoder_outputs, attention_weight, mask

### 2. 共情对话模型构建

接下来，将详细介绍为适应本项目的共情对话场景，应该如何对原始 Transformer 模型进行改动，使得 Transformer 模型可以「识别情感⇢表达情感⇢生成共情回复」。

#### 2.1 原始 Transformer 回顾  

1. Embedding + Positional Encoding  
   - 对每个输入 token 转为向量 $e_i\in\mathbb R^d$，再加上正余弦位置编码 $\mathrm{PE}(i)$：  
   $$x_i = e_i + \mathrm{PE}(i)$$
   - 代码：  
      ```python
      x = self.input_dropout(inputs)
      x = self.embedding_proj(x)
      x += self.timing_signal[:, :seq_len, :]
      ```  
2. 多头自注意力 (Self-Attention)  
   - 对序列 ${x_i}$ 计算：  
     $$\mathrm{Attn}(Q,K,V)=\mathrm{softmax}\bigl(\tfrac{QK^T}{\sqrt{d_k}}+\mathrm{Mask}\bigr)V$$
   - MultiHead 会并行执行 $h$ 次，再拼接、线性映射回维度 $d$。  
3. 前馈网络 (FFN)  
   - 对每个位置独立执行  
     $\mathrm{FFN}(x)=\max(0, xW_1+b_1)W_2+b_2.$  
4. Encoder×N 层 与 Decoder×N 层  
   - Encoder 层堆叠自注意力 + FFN；Decoder 第一层做因果屏蔽自注意力，第二层做跨注意力。

#### 2.2 情感预置

我们通过采用共享设计的词嵌入，并使用额外的“情感掩码嵌入”与上下文嵌入相加，实现“情感前置/预置”的效果。

`NRCDict.json` 是存放在 `datasets/empathetic-dialogue/` 下的一个外部情感词典，其内容是一个 JSON 数组，第一项（`[0]`）即是一个情感词汇表列表。模型在启动时会加载该情感词典：

```python
EMODICT = json.load(open('datasets/empathetic-dialogue/NRCDict.json'))[0]
```

然后，构造情感提示 ID 序列 `mask_context`，对数据集上下文中的每个词，若在 EMODICT 中，则对应位置设置一个「提示 ID」（>0），否则为 0。

通过叠加情感提示嵌入，对特定「情感词」位置注入可学习偏置，引导后续注意力与表示把这些位置看得更重要。
$$
\underbrace{\mathrm{emb}(c_i)}_{\text{上下文嵌入}} \;+\;
\underbrace{\mathrm{emb}(\mathrm{mask\_context}_i)}_{\text{情感提示嵌入}}
\;\to\; x_i
$$

代码：
```python
# 取自 baselines/EmoPrepend.py 的核心调用思路
self.embedding = share_embedding(self.vocab, config.pretrain_emb)
...
mask_src = enc_batch.data.eq(config.PAD_idx).unsqueeze(1)  # (bsz, 1, src_len)
emb_mask = self.embedding(batch["mask_context"])         # 由外部构造的情感提示 ids → 嵌入
src_emb = self.embedding(enc_batch) + emb_mask            # 叠加到上下文嵌入上
encoder_outputs = self.encoder(src_emb, mask_src)
```

要点：
- `share_embedding`：提供共享的 `nn.Embedding`（可与生成头权重共享）。
- `mask_context`：与 `enc_batch` 等长的“提示序列”，在需要强调的 token 位置（如情感词、线索词）注入可学习的偏置，形成“情感引导”。
- `src_emb = embedding(context) + embedding(mask)`：简单相加即可在编码层面实现引导，无需改动注意力形态。

此外，在训练或分析时，也会用 get_emotion_words 提取生成句中的情感关键词：
```python
def get_emotion_words(utt_words):
    """
    从情感词典中获取与输入词列表匹配的情感词
    参数:
        utt_words (list[str]): 分词后的句子词列表
    返回:
        emo_ws (list[str]): 匹配到的情感词列表
    """
    emo_ws = []
    for u in utt_words:
        if u in EMODICT:
            emo_ws.append(u)
    return emo_ws
```

#### 2.3 情感分类头

1. 全局情感特征提取  
   - 类似 BERT 的 [CLS]：取 `encoder_outputs[:,0]` 作为整句聚合向量 $q_h$。  
2. 线性映射到情感类别  
    $$
    \ell = W_{\text{emo}}\,q_h + b_{\text{emo}},\quad
    p(\text{emo})=\mathrm{softmax}(\ell)
    $$ 
    代码：

    ```python
    q_h = encoder_outputs[:, 0]
    emotion_logit = self.decoder_key(q_h)
    loss_emotion = nn.CrossEntropyLoss(reduction='sum')(emotion_logit, batch['emotion_label'])
    pred_emotion = np.argmax(emotion_logit.detach().cpu().numpy(), axis=1)
    ...
    loss = self.criterion(logit.contiguous().view(-1, logit.size(-1)), dec_batch.contiguous().view(-1))
    loss += loss_emotion
    loss += loss_from_d
    ```
3. 多任务联合训练
    $$
    \mathcal{L}=\mathcal{L}_{\mathrm{gen}}+\lambda_{\mathrm{emo}}\mathcal{L}_{\mathrm{emotion}}
    \quad(\,+\mathcal{L}_{\mathrm{adv}}\;\text{可选})
    $$
    - 使编码器既要学习生成信号，也要捕捉情感信号；梯度同时来自生成与分类，增强情感感知。
    - 情感辅助任务督促编码器关注情感信号，与“情感预置”的引导相互补充。

#### 2.4 指针-生成器

为兼顾“复制源文本片段”和“生成词表内词”，`EmoPrepend` 使用门控 `p_gen` 在两种分布间进行加权并融合到扩展词表：

1. 生成分布 vs. 复制分布  
    - 生成门：  
    $$p_{\text{gen}}=\sigma(w_p^T x_t + b_p),\quad \alpha_t=p_{\text{gen}}$$  
    - 常规词表分布  
    $$P_{\mathrm{vocab}}=\mathrm{softmax}(W_o x_t / T)$$  
    - 注意力复制分布  
    $$P_{\mathrm{copy}} = \mathrm{softmax}(\mathrm{attn\_scores})$$  
2. 融合
    $$
    P(w)=\alpha_t\,P_{\mathrm{vocab}}(w)\;
        +\;(1-\alpha_t)\sum_{i:x_i=w}P_{\mathrm{copy}}(i).
    $$

代码：
```python
# 线性变换得到生成门（与上下文有关）
p_gen = torch.sigmoid(self.p_gen_linear(x))              # (B, T, 1)

# 常规词表分布与注意力分布（经 softmax 温度）
vocab_dist = softmax(Wx/temp)                            # (B, T, V)
attn_dist  = softmax(attn/temp)                          # (B, T, S)

# 门控融合
a = p_gen * vocab_dist                                   # 生成部分
b = (1 - p_gen) * attn_dist                              # 复制部分

# 将注意力分布散射到“扩展词表”索引位置（OOV 统一映射到扩展位置）
logit = log(a.scatter_add(2, enc_batch_extend_vocab, b))
```

要点：
- `enc_batch_extend_vocab`：把每个源 token 在“扩展词表”中的索引提供给 `scatter_add`，把注意力概率累加到正确的词位。
- 复制能力对于 OOV、专有名词、长尾表达尤为关键，是对话与摘要类任务常用增强。

#### 2.5 训练细节与学习率调度  
1. Label Smoothing（可选）  
   $$
   \mathcal{L}_{\mathrm{LS}}=-\sum (1-\epsilon)y\log p+( \epsilon/(V-1))\sum_{y'\neq y}\log p(y').
   $$
2. NoamOpt 学习率策略  
   $$
   \mathrm{lr} \propto d^{-0.5}\min(\text{step}^{-0.5},\;\text{step}\times \mathrm{warmup}^{-1.5}).
   $$
   - 保证前期快速上升，后期平稳收敛。


通过上面几个模块的无缝对接，EmoPrepend 在输入端主动「标记情感」、中间端以「分类头」监督情感语义、输出端以「指针-生成」复合分布，最终让模型既能正确捕捉对话情感，又能在生成时自如复现、共情反馈。

我们可以实例化完整的模型来输出查看模型的结构和参数统计。

In [15]:
import torch
import torch.nn as nn
import sys
import os

if hasattr(sys, 'argv') and any('kernel' in arg for arg in sys.argv):
    # 在Jupyter环境中，清理sys.argv以避免argparse冲突
    original_argv = sys.argv.copy()
    sys.argv = [sys.argv[0]]  # 只保留脚本名称

from utils.data_loader import prepare_data_seq
from utils.common import *
from Model.EmoPrepend import EmoP
from utils import config

def count_parameters(model):
    """统计模型参数数量"""
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total_params, trainable_params

def print_model_structure(model, show_templates=True):
    """打印适中详尽的模型结构摘要，兼顾信息量与输出长度"""
    print("=" * 60)
    print("EmoPrepend 结构摘要")
    print("=" * 60)

    total_params, trainable_params = count_parameters(model)
    print(f"参数总量: {total_params:,}  |  可训练: {trainable_params:,}")
    print(f"词汇表: {model.vocab_size}  |  嵌入维度: {config.emb_dim}")
    print(f"编码层数: {config.hop}  |  隐藏维度: {config.hidden_dim}  |  头数: {config.heads}")
    print(f"情感类别数: {model.decoder_key.out_features}")

    flags = []
    if getattr(config, 'pointer_gen', False):
        flags.append("Pointer-Gen")
    if getattr(config, 'weight_sharing', False):
        flags.append("WeightSharing")
    if getattr(config, 'noam', False):
        flags.append("NoamLR")

    if getattr(config, 'noam', False):
        optim_str = "Noam 调度器"
    else:
        optim_str = f"Adam(lr={getattr(config, 'lr', 'N/A')})"
    print(f"优化器: {optim_str}")

    def module_params(m):
        return sum(p.numel() for p in m.parameters())

    print("-" * 60)
    print("参数分布（按主要模块）")
    print(f"- Embedding:   {module_params(model.embedding):,}")
    print(f"- Encoder:     {module_params(model.encoder):,}")
    print(f"- Decoder:     {module_params(model.decoder):,}")
    print(f"- Generator:   {module_params(model.generator):,}")
    print(f"- EmotionHead: {module_params(model.decoder_key):,}")

    if show_templates:
        print("-" * 60)
        print("结构摘要（首层模板）")
        # Embedding
        try:
            emb_shape = tuple(model.embedding.lut.weight.shape)
            print(f"- Embedding 权重: {emb_shape}")
        except Exception:
            print("- Embedding 权重: 未知")

        # Encoder
        print(f"- Encoder 层数: {getattr(config, 'hop', 'N/A')}")
        enc0 = None
        try:
            enc0 = model.encoder.enc[0]
        except Exception:
            try:
                enc0 = model.encoder.enc
            except Exception:
                enc0 = None
        if enc0 is not None:
            print("  Encoder 单层模板:")
            child_names = [n for n, _ in enc0.named_children()]
            print(f"  - {enc0.__class__.__name__}")
            print(f"    子模块: {', '.join(child_names)}")
        else:
            print("  Encoder 单层模板: 未获取")

        # Decoder
        try:
            dec0 = model.decoder.dec[0]
        except Exception:
            try:
                dec0 = next(iter(model.decoder.dec))
            except Exception:
                dec0 = None
        print(f"- Decoder 层数: {getattr(config, 'hop', 'N/A')}")
        if dec0 is not None:
            print("  Decoder 单层模板:")
            child_names = [n for n, _ in dec0.named_children()]
            print(f"  - {dec0.__class__.__name__}")
            print(f"    子模块: {', '.join(child_names)}")
        else:
            print("  Decoder 单层模板: 未获取")

        # Generator & Emotion head
        print(f"- Generator: Linear({getattr(config, 'hidden_dim', 'H')} -> {model.vocab_size})")
        print(f"- EmotionHead: Linear({getattr(config, 'hidden_dim', 'H')} -> {model.decoder_key.out_features})")

def create_emoprepend_model():
    """创建并返回EmoPrepend模型"""
    class SimpleVocab:
        def __init__(self):
            self.n_words = 10000
            self.word2index = {}
            self.index2word = {}
        
    vocab = SimpleVocab()
    program_number = 32  # 默认情感类别数
    
    # 实例化模型
    model = EmoP(
        vocab=vocab,                    # 词汇表对象
        decoder_number=program_number,  # 情感类别数（32）
        is_eval=False,                  # 是否为评估模式
        load_optim=False                # 是否加载优化器状态
    )
    
    # 移动到指定设备
    model.to(config.device)
    
    print("模型创建完成!")
    print()
    
    return model, vocab, program_number


model, vocab, program_number = create_emoprepend_model()

# 输出模型结构
print_model_structure(model)

print(f"device={getattr(config, 'device', 'cpu')}, emb_dim={getattr(config, 'emb_dim', 'N/A')}, hidden_dim={getattr(config, 'hidden_dim', 'N/A')}, hop={getattr(config, 'hop', 'N/A')}, heads={getattr(config, 'heads', 'N/A')}")


模型创建完成!

EmoPrepend 结构摘要
参数总量: 3,049,519  |  可训练: 3,049,519
词汇表: 10000  |  嵌入维度: 100
编码层数: 6  |  隐藏维度: 100  |  头数: 1
情感类别数: 32
优化器: Adam(lr=0.0001)
------------------------------------------------------------
参数分布（按主要模块）
- Embedding:   1,000,000
- Encoder:     409,506
- Decoder:     626,712
- Generator:   1,010,101
- EmotionHead: 3,200
------------------------------------------------------------
结构摘要（首层模板）
- Embedding 权重: (10000, 100)
- Encoder 层数: 6
  Encoder 单层模板:
  - EncoderLayer
    子模块: multi_head_attention, positionwise_feed_forward, dropout, layer_norm_mha, layer_norm_ffn
- Decoder 层数: 6
  Decoder 单层模板:
  - DecoderLayer
    子模块: multi_head_attention_dec, multi_head_attention_enc_dec, positionwise_feed_forward, dropout, layer_norm_mha_dec, layer_norm_mha_enc, layer_norm_ffn
- Generator: Linear(100 -> 10000)
- EmotionHead: Linear(100 -> 32)
device=cpu, emb_dim=100, hidden_dim=100, hop=6, heads=1
